In [1]:
import os
import sys
from time import sleep
import json
import pandas as pd
import os
import gc
import numpy as np
from tqdm.notebook import tqdm
import pickle

root = "../input/meta-kaggle"
santa_id = 24539

episode_agents = pd.read_csv(os.path.join(root, "EpisodeAgents.csv"))
agents = pd.read_csv(os.path.join(root, "Episodes.csv"))

santa_agents = agents.loc[(agents['CompetitionId'] == santa_id)].reset_index(drop = True)
gc.collect()

#get episode of santa competition
episode_agents = episode_agents.merge(santa_agents, how = 'inner', left_on = 'EpisodeId', right_on = "Id")
gc.collect()

0

In [2]:
keep_col = [
    "EpisodeId", "Index", "SubmissionId", "EndTime", "Type", 'UpdatedScore'
]

episode_agents = episode_agents[keep_col]

#discard validation episode
episode_agents = episode_agents[episode_agents['Type'] == 1].reset_index(drop = True)

#convert time column to dt
episode_agents['EndTime'] = pd.to_datetime(episode_agents['EndTime'].transform(lambda x: x[:16]))

#get last episode for each submission (last score on leaderboard) and sort by score
last_submission_update = episode_agents.sort_values(['SubmissionId', 'EndTime'], ascending = False).groupby('SubmissionId').head(1)

In [3]:
episode_agents = episode_agents[["EpisodeId", "Index", "SubmissionId"]]
last_submission_update = last_submission_update[['SubmissionId', 'UpdatedScore']]

del agents, santa_agents
gc.collect()

20

In [4]:
episode_agents.to_pickle('episode_agents.pkl')
last_submission_update.to_pickle('last_submission_update.pkl')